In [ ]:
// Importieren der benötigten Module
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE, 0.0.1"
#r "nuget: FSharp.Stats"

// ... und öffnen der benöätigten Module
open FsODE
open Plotly.NET
open FSharp.Stats


// der Rechenalgorithmus der Programmbibliothek, die wir benutzen:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

### Einfaches Model vom Lac Operon

Also Intracellular lactose gegen die Zeit
Dabei wird Intracellular lactose postiv reguliert

Differenzialgleichung

let intralac' = alpha + beta * (intalac**2/(K_d**2. + intalac**2))-gamma*intalac

Funktioniernde Parameter:

alpha = 0.5
beta = 10.
K_d = 4.

Bistabilität einführen (das folgende ist nur eine Kopie vom alten tag 3, kann gerne komplett verändert werden) <br>


Ein System das zwei stabile Fixpunkte hat wird auch istabiles System genannt. **Bistabilität** ist ein der Schlüsseleigenschaften der meisten biologischen Systeme, da das System auch dann an einem der beiden Fixpunkte verweilt, auch wenn es leichte Störungen oder leicht veränderte äußere Einflüße gibt. Das bedeutet ein biologisches System das sich um den Schutz gegen stark gestiegene Temperaturen kümmert und die dafür beötigten Proteine produizert wird nicht aktiv wenn sich die Temperatur nur leicht ändert, sondern erst wenn es zu einer starken Abweichung von den normalern Bedingungen kommt. Dies ermöglicht einen effizenteren Einsatz von Resourcen, es kommt nur dann zu einer Proteinproduktion wenn diese benötigt werden, und damit auch eine bessere Überlebenswahrscheinlichkeit. <br>
Bistabilität ist allerdings nur eine spezialform der **Multistabilität**. Multistabilität bedeutet das ein System mehere Fixpunkte hat zu denen es wandern kann, anhängig von den äußeren Einflüßen. 


-----------------

Muss kompleet überarbeitet werden (anderes Model für ultrasensitives Verhlaten, Überleitung vom Lac operon )

### Hill Funktion

Allerdings ist es an dieser Stelle relevant eine andere Darstellung einzuführen. Die meisten Antworten in der Genregulkation und bei Protein-Protein-Interaktionen haben ein **ultrasensitives** Verhalten. Das heißt das ein Prozess nur dann stattfindet wenn ein bestimmter Schwellenwert überschritten wird, während es davor keinen Effekt gibt. Oft ist dieses ultrasensitive Verhalten bedingt durch **Kooperativität**. Ein Beispiel dafür wäre Hemoglobin. Hemoglobin bindet bis zu vier Moleküle Sauerstoff. Wenn kein Molekül Sauerstoff gebunden ist, befindet sich das Hemoglobin in dem sogenannten T-State. In diesem ist die Affinität von Hemoglobin für Sauerstoff sehr gering und eine Bindung von Sauerstoff ist eher unwahrscheinlich. Fall aber ein Sauerstoffmolekül dennoch an das Hemoglobin bindet kommt es zu einer Konformationsänderung. Durch diese Änderung wird die Affinität von Hemoglobin für Sauerstoff deutlich erhöht und es ist wahrscheinlich das drei weitere Sauerstoffmolekükle an das Hemoglobin binden. Sobald alle vier Sauerstoffmoleküle an das Hemoglobin gebunden haben wird von dem sogenannten R-State gesprochen.<br>
Das Verhalten kann daher als ultrasensitiv beschrieben werden, weil es nur bei hoch genugen Sauerstoffkonzentration zu der Bildung des R-states kommt. Bei niedrigen Konzentration bnindet zwar ab und zu ein Molekül Sauerstoff an das Hemoglobin, aber weitere Sauerstoffmoleküle sind zu weit entfernt um die jetzt höhere Affinität auszunutzen, weshalb das gebunde Sauerstoff nach einiger Zeit das Hemoglobin wieder verlässt.<br>
Die beschreibung von solchen Ultrasensitiven Prozessen kann mit der sogenannten Hillfunktion beschrieben werden. Das anwenden von dieser auf die bisher besprochenen negative Autoregulation kann generisch mit der folgenen Differenzialgleichung beschrieben werden 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ (\frac {Proteinkonzentration}{K_{D}})^{n}}  - \gamma*Proteinkonzentration$$

die Gleichung für eine ultrasentives Verhalten von einer Proteinproduktion die durch einen Repressor beeinflusst wird ist identisch, nur müssen sie die Proteinkonzentration im Term, der $\beta$ beeienflusst, durch die Repressorkonezntration ersetzten. Die DIfferenzialgleichung für postive Autoregulation ist 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {Proteinkonzentration^{n}}{{(K_{D})^{n}} + Proteinkonzentration^{n}} - \gamma*Proteinkonzentration$$
auch hier führt das ersetzten der Protienkonzentration, im $\beta$ beeinflussenden Term, mit der Aktivatorkonezentration zu der Gleichung für eine Proteinproduktion die von diesem beieinflusst wird. 
Der Paramerter $n$ ist der Hillkoeffizent und beschreibt wie stark das ultrasensitve Verhalten ist. Ein Wert von 1 beschreibt die bisher besprochenen Verhaltensweisen, während ein Wert von 2 oder höher das ulatrsensitive Verhalten beschreibt. 

Aufgabe:

Setzen sie die Gleichung für postive Autoregulation in die nachfolgende Simulation ein, dabei sollte die postive Autoregulation durch eine Hill Funktion beschrieben werden (Bitte beachten sie, dass sie die Proteinkonznetration als "Proteinkonznetration2" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      2.0  // Hill Koeffizent 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonznetration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonznetration2 = P[1] // Änderung in der Proteinmenge
        //Änderung der Proteinkonzentration ohne positive Autoregulation
        let Proteinkonznetration1' = beta - (gamma*Proteinkonznetration1)
        // Änderung des Proteinkonzentration bei positiver Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonznetration2' = beta * (Proteinkonznetration2**n)/((Proteinkonznetration2**n) + (K_d**n)) - (gamma*Proteinkonznetration2)

        [| Proteinkonznetration1'; Proteinkonznetration2' |]

//Startkonzentrationen
let P0 = [| 
    //Startkonzentration für das unregulierte Protein (muss nicht verändert werden)
    0.0 
    //Startkonzentration für das durch postive Autoreguliation regulierte Protein (hier bitte die Startwerte ändern)
    4.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("negative AR");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

Die Konstanten für den gezeigten Graphen sind:
$$\beta = 10$$
$$\gamma= 1$$
$$n = 1$$
$$K_{D} = 1$$
$$Proteinkonzentration (0) = 0.1$$
Aufgabe: <br>
Welchen Effekt hat postive Autoregulation bei einem Wert von $n=1$ auf die Antwortszeit?


Hier die Antwort eintippen

Aufgabe <br>
Was passiert bei $n=2$?
Kopieren sie die obenstehnden code block für die postive Autoregulation und fügen sie diesen in den folgenden Code Block ein. Verändern sie dann die Hill Konstante. 

In [ ]:
//Code block einfügen

Was beobachten sie, welchem bereits bekannten Effekt ähnelt dies?

Hier die Antwort eintippen

Die Punktattrkationen für dieses Model sind: <br>
stabil: 0 und 8 <br>
instabil: 2

Aufgabe: <br>
Teilen sie Gleichung welche die Proteinproduktion in Abhängigkeiten von ultrasenistver postiver Auporegulation beschreibt in zwei Gleichungen. Die erste soll die Produktion des Proteins beschreiben, die zweite die Abnahame, also
$$ \frac {dProteinkonzentration} {dt} = \textrm{Produktion des Proteins} $$
$$ \frac {dProteinkonzentration} {dt} = \textrm{Verringerung der Proteinkonzentration}$$

Fügen sie die beiden Gleichungen in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{d}$ als K_d eingesetzt wird). 

In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      2.0  // Hill Koeffizent 

//Graphische Darstellung der Gleichungen
[
    // Fügen sie die Gleichung für den Proteinabbau ein
    Chart.Line [ for x in 0.0 .. 0.2 .. 15.0 -> (x, (gamma*x)) ]
    |> Chart.withTraceInfo("Proteinabbau");
    //Fügen sie die Gleichung für die Proteinproduktion ein
    Chart.Line [ for x in 0.0 .. 0.2 .. 15.0 -> (x, beta * (x**n)/((x**n) + (K_d**n))) ]
    |> Chart.withTraceInfo("Proteinproduktion");
]

|> Chart.combine
|> Chart.withXAxisStyle("Proteinkonzentration")
|> Chart.withYAxisStyle("Veränderung der Proteinkonzentration")

Ordentlich schreiben...

Der gezeigte Graph zeigt wie sich Produktion und Verringerung verhalten. Die Produktion ist eine Sigmodilade Kurve, während die Verringerung einfach nur eine Gerade ist (wie bei den  Gleichung zu erwarten). Die Stellen an denen sich die beiden Graphen schneiden sind somit die Punkte bei denen Produktion und Verringerung der Proteinproduktion übereinstimmen. Das heißt an diesen Punkten heben sich beide auf und es handelt sich somit um Punktattraktionen. Wenn sie die Werte hier ablesen und mit denen von der Simulation vergleichen sollten sie sehen, das sie identisch sind. SIe können dieses Diagramm nutzen um herauszufinden wohin bestimme Werte von $Proteinkonzentration (0)$ wandern werden.
Dieses Diagramm macht es zudem sehr einfach zu bestimmen ob es sich um stabile oder instabile Punktattraktionen handelt. 
...

Aufgabe: <br>
Kopieren sie die obenstehnden code block und fügen sie diesen in den folgenden Code Block ein. 


Varrieren sie die Werte für die Abbaukonsante $\gamma$. Bestimmen sie bei welchen Werten von $\gamma$ es zur Bistabilität kommt und bei welchen nicht. Bestimmen sie außerdem ob es sich um stabile oder instabile Punkte handelt. Sinnvolle Werte für $\gamma$ liegen im Bereich von 0 bis 10.


Notieren sie hier die Werte

Aufgabe: <br>
Setzen sie jetzt die Werte in den nachfolgenden Code Block ein und führen sie diesen anschliesend aus.


In [ ]:
// Werte
//code


Dieser Graph ist das sogenannte Bifurkationsdiagramm. 

Erklärung Bifurkation.

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen.

------------------

### Osszillation/Hopf Bifukration folgt